In [1]:
import pickle
import random

In [2]:
strategy = pickle.load(open("perfectPolicy.p", "rb"))

In [3]:
strategy 


{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 (1, 0, 0, 0, 0, 0, 0, 0, 0): array([0., 0., 0., 1., 0., 0., 0., 0.]),
 (1, 2, 0, 0, 0, 0, 0, 0, 0): array([0., 0., 1., 0., 0., 0., 0.]),
 (1, 2, 1, 0, 0, 0, 0, 0, 0): array([0., 1., 0., 0., 0., 0.]),
 (1, 2, 1, 2, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0.]),
 (1, 2, 1, 2, 1, 0, 0, 0, 0): array([1., 0., 0., 0.]),
 (1, 2, 1, 2, 1, 2, 0, 0, 0): array([1., 0., 0.]),
 (1, 2, 1, 2, 1, 2, 0, 1, 0): array([1., 0.]),
 (1, 2, 1, 2, 1, 2, 2, 1, 0): array([1.]),
 (1, 2, 1, 2, 1, 2, 0, 1, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 2, 0, 0): array([0., 0., 1.]),
 (1, 2, 1, 2, 1, 1, 2, 0, 0): array([0., 1.]),
 (1, 2, 1, 2, 1, 1, 2, 2, 0): array([1.]),
 (1, 2, 1, 2, 1, 1, 2, 0, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 2, 1, 0): array([0., 1.]),
 (1, 2, 1, 2, 1, 0, 2, 1, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 0, 2, 0): array([1., 0., 0.]),
 (1, 2, 1, 2, 1, 1, 0, 2, 0): array([1., 0.]),
 (1, 2, 1, 2, 1, 1, 0, 2, 2): array([1.]),
 (1, 2, 1, 

In [4]:
len(strategy)

4520

In [5]:
def tictactoe():
    {
        
    }

In [ ]:
def generate_board():
    return (0, 0, 0, 0, 0, 0, 0, 0, 0)

In [ ]:
def current_player(board):
    return 1 if board.count(1) == board.count(2) else 2

In [6]:
def actions():
    {
        
    }

In [7]:
def random_strategy():
    return random.choice(actions())

In [8]:
def policy_strategy():
    {
        
    }

In [9]:
def perfect_strategy():
    
    return strategy
    

In [11]:
def check_winner():
    {
        
    }

In [ ]:
def training_loop():
    {
        
    }